# Hyperthyroid Disease Classification
### Group 17: Matthew Gillies, Ryan Lee, Eric Liu, Arman Moztarzadeh

## Summary
In this report, we trained a classifier to predict the presence of hyperthyroidism using varying attributes such as age, sex, prior treatment for thyroid disease, and amount of thyroid hormones in the body. Our classifier has a 98% accuracy rate on the test set.

## Introduction
The **Thyroid Disease dataset** obtained from the **UCI Machine Learning Repository** will be used to predict the presence of hyperthyroidism. Hyperthyroidism is an issue that occurs when the thyroid gland produces an excess amount of thyroid hormones (De Leo et al., 2016). As a result, the body's metabolism greatly "speeds up", resulting in weight loss, rapid heartbeat, fatigue, shaky hands, sweating, and more (U.S. Department of Health and Human Services, n.d.). 
<br>
Studies have shown that within the population, certain groups are more predisposed to getting hyperthyroidism. Hyperthyroidism is more common in women, women who were recently pregnant, those with type 1 or type 2 diabetes, alongside other factors (Allahabadia, 2000). In order to help predict whether someone has hyperthyroidism, we are using 19 attributes provided from the Thyroid Disease dataset. The attributes we are using are age, sex, if they take medication for thyroid disease (thyroxine or antithyroid medication), pregnant status, prior treatment for thyroid disease (thyroid surgery or Ii131 radiotherapy for hyperthyroidism), and amount of different hormones in the body (TSH, TT4, T4U, and FTI). **With these factors, we are hoping to make an accurate classifier in to predict whether or not someone has hyperthyroidism**

## Methods and Results

### Data-Cleaning
To begin this analysis we read in the data from the original source, merging **solely** for pre-processing. The `binaryClass` feature is manipulated so that all values are either positive or negative, removing all other diagnoses. Next, we replace all `?` values with `NaN` values, and removed all columns that were irrelevant to our classification model or had an extremely large amount of `NaN` values. We then remove all of the remaining rows with NA values from the data set. Since all of the columns are of datatype `Object`, they are converted to their respective data-types (either numeric or categorical). Following this we converted the `binaryClass` column from character values ("P" and "N") to the reverse in boolean integer values ("0" and "1" respectively) to match the classification model. In this data set positive labels actually represented a negative diagnosis.
### Exploratory Data Analysis
After data-cleaning we preformed exploratory data analysis (EDA) through summary statistics, correlations of numeric features, and value counts for the entire data set. 
### Model Training
We split the data into training and testing set with a 70/30 split, while also separating the `binaryClass` feature (**target**) from the rest of the data set. Following this a `ColumnTransformer` was created to scale all numeric variables and one-hot-encode all categorical variables to ensure they are in a state that the model can process. The `ColumnTransformer` was then fitted and transformed on the training set, along with transforming the test set. We then performed cross-validation with a LogisticRegression model on the training data, which returned an average validation score of 98%. Applying the model again to the full training set, we again produced a 98% accuracy score. Furthermote, we visualized the training predictions with `TSH` and `TT4` concentrations on the x and y axes respectively. Finally, **The model was applied to the test set, which produced a 99% accuracy**. Another plot visualizing the class predictions were created along with a confusion matrix to realize the impact of the predictions. From the confusion matrix we can see that most of our incorrect predictions are false negatives, which are not preferable as an incorrect positive disease diagnose would have less consequential impact than an incorrectly predicting some as disease free.

## Imports

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import make_pipeline
import seaborn as sn
import warnings
warnings.filterwarnings('ignore')

## Data-Cleaning

In [2]:
if not os.path.exists("data"):
    os.makedirs("data")
path1 = 'http://archive.ics.uci.edu/ml/machine-learning-databases/thyroid-disease/allhyper.data'
path2 = 'http://archive.ics.uci.edu/ml/machine-learning-databases/thyroid-disease/allhyper.test'
columnNames = ["age", "sex", "on thyroxine", "query on thyroxine",
               "on antithyroid medication", "sick", "pregnant",
               "thyroid surgery", "I131 treatment", "query hypothyroid",
               "query hyperthyroid", "lithium", "goitre", "tumor",
               "hypopituitary", "psych", "TSH measured", "TSH", "T3 measured",
               "T3", "TT4 measured", "TT4", "T4U measured", "T4U",
               "FTI measured", "FTI", "TBG measured", "TBG", "referral source",
               "binaryClass"]
dfData = pd.read_csv(path1, names=columnNames)
dfTest = pd.read_csv(path2, names=columnNames)
hyperthyroid_df = pd.concat([dfData, dfTest])

In [3]:
# Dataset is backwards so P refers to negative and N refers to positive
# Other if statements are to remove various other labels for binary class
for row in range(len(hyperthyroid_df)):
    if (hyperthyroid_df['binaryClass'].values[row][0] == "n"):
        hyperthyroid_df['binaryClass'].values[row] = "P"
    if (hyperthyroid_df['binaryClass'].values[row][0] == "h"):
        hyperthyroid_df['binaryClass'].values[row] = "N"
    if (hyperthyroid_df['binaryClass'].values[row][0] == "T"):
        hyperthyroid_df['binaryClass'].values[row] = "?"
    if (hyperthyroid_df['binaryClass'].values[row][0] == "g"):
        hyperthyroid_df['binaryClass'].values[row] = "?"
    if (hyperthyroid_df['binaryClass'].values[row][0] == "s"):
        hyperthyroid_df['binaryClass'].values[row] = "?"
hyperthyroid_df.to_csv('data/hyperthyroid.csv')

In [4]:
# Reading in data
# hyperthyroid_df = pd.read_csv("data/hyperthyroid.csv")
hyperthyroid_df.head()

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,...,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,binaryClass
0,41,F,f,f,f,f,f,f,f,f,...,t,125,t,1.14,t,109,f,?,SVHC,P
1,23,F,f,f,f,f,f,f,f,f,...,t,102,f,?,f,?,f,?,other,P
2,46,M,f,f,f,f,f,f,f,f,...,t,109,t,0.91,t,120,f,?,other,P
3,70,F,t,f,f,f,f,f,f,f,...,t,175,f,?,f,?,f,?,other,P
4,70,F,f,f,f,f,f,f,f,f,...,t,61,t,0.87,t,70,f,?,SVI,P


In [5]:
hyperthyroid_df.shape

(3772, 30)

In [6]:
# Replacing ? values with NA
hyperthyroid = hyperthyroid_df.replace("?", np.nan)

In [7]:
hyperthyroid.isna().sum()

age                             1
sex                           150
on thyroxine                    0
query on thyroxine              0
on antithyroid medication       0
sick                            0
pregnant                        0
thyroid surgery                 0
I131 treatment                  0
query hypothyroid               0
query hyperthyroid              0
lithium                         0
goitre                          0
tumor                           0
hypopituitary                   0
psych                           0
TSH measured                    0
TSH                           369
T3 measured                     0
T3                            769
TT4 measured                    0
TT4                           231
T4U measured                    0
T4U                           387
FTI measured                    0
FTI                           385
TBG measured                    0
TBG                          3772
referral source                 0
binaryClass   

In [8]:
# Due to large amount of NA values in TBG and T3 these features will be removed from the dataset. 
# All "measured" features will also be removed as once NA's are removed they will all be "t"
# We will also removed the referral source column as it is quite irrelevant
# All other NA rows will simply be dropped
hyper = hyperthyroid.drop(columns=["TBG", "TBG measured", "T3", "T3 measured", "TSH measured",
                                   "TT4 measured", "FTI measured", "T4U measured", "referral source"])

In [9]:
hyper_clean = hyper.dropna()

In [10]:
hyper_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3085 entries, 0 to 971
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   age                        3085 non-null   object
 1   sex                        3085 non-null   object
 2   on thyroxine               3085 non-null   object
 3   query on thyroxine         3085 non-null   object
 4   on antithyroid medication  3085 non-null   object
 5   sick                       3085 non-null   object
 6   pregnant                   3085 non-null   object
 7   thyroid surgery            3085 non-null   object
 8   I131 treatment             3085 non-null   object
 9   query hypothyroid          3085 non-null   object
 10  query hyperthyroid         3085 non-null   object
 11  lithium                    3085 non-null   object
 12  goitre                     3085 non-null   object
 13  tumor                      3085 non-null   object
 14  hypopitui

In [11]:
# Changing Dtype of the columns to numeric/categorical
num_cols = ['age', 'TSH', 'TT4', 'T4U', 'FTI']
cat_cols = ['sex', 'on thyroxine', 'query on thyroxine', 'on antithyroid medication',
            'sick', 'pregnant', 'thyroid surgery', 'I131 treatment', 'query hypothyroid',
            'query hyperthyroid', 'lithium', 'goitre', 'tumor', 'psych', 'binaryClass', 'hypopituitary']
hyper_clean[num_cols] = hyper_clean[num_cols].apply(pd.to_numeric,
                                                    errors='coerce', axis=1)
for col in cat_cols:
    hyper_clean[col] = hyper_clean[col].astype('category')

In [12]:
# Changing binaryClass so 0 represents negative and 1 represents positive
hyper_clean['binaryClass'] = hyper_clean['binaryClass'].replace(["N", "P"], [1, 0])

In [13]:
hyper_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3085 entries, 0 to 971
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   age                        3085 non-null   float64 
 1   sex                        3085 non-null   category
 2   on thyroxine               3085 non-null   category
 3   query on thyroxine         3085 non-null   category
 4   on antithyroid medication  3085 non-null   category
 5   sick                       3085 non-null   category
 6   pregnant                   3085 non-null   category
 7   thyroid surgery            3085 non-null   category
 8   I131 treatment             3085 non-null   category
 9   query hypothyroid          3085 non-null   category
 10  query hyperthyroid         3085 non-null   category
 11  lithium                    3085 non-null   category
 12  goitre                     3085 non-null   category
 13  tumor                      3085 no

In [14]:
hyper_clean.describe()

,age,TSH,TT4,T4U,FTI
count,3085.000000,3085.000000,3085.000000,3085.000000,3085.00000
mean,52.821718,4.982172,108.694814,0.995246,110.31154
std,20.099428,23.975905,35.763281,0.194844,32.79471
min,1.000000,0.005000,2.000000,0.250000,2.00000
25%,37.000000,0.480000,88.000000,0.880000,93.00000
50%,55.000000,1.300000,104.000000,0.980000,107.00000
75%,68.000000,2.600000,125.000000,1.080000,124.00000
max,455.000000,530.000000,430.000000,2.320000,395.00000


In [15]:
for c in hyper_clean.columns:
    print("---- %s ---" % c)
    print(hyper_clean[c].value_counts())

---- age ---
70.0    80
59.0    79
60.0    76
73.0    70
55.0    70
        ..
4.0      1
6.0      1
10.0     1
92.0     1
11.0     1
Name: age, Length: 93, dtype: int64
---- sex ---
F    2096
M     989
Name: sex, dtype: int64
---- on thyroxine ---
f    2683
t     402
Name: on thyroxine, dtype: int64
---- query on thyroxine ---
f    3056
t      29
Name: query on thyroxine, dtype: int64
---- on antithyroid medication ---
f    3053
t      32
Name: on antithyroid medication, dtype: int64
---- sick ---
f    2956
t     129
Name: sick, dtype: int64
---- pregnant ---
f    3043
t      42
Name: pregnant, dtype: int64
---- thyroid surgery ---
f    3037
t      48
Name: thyroid surgery, dtype: int64
---- I131 treatment ---
f    3034
t      51
Name: I131 treatment, dtype: int64
---- query hypothyroid ---
f    2877
t     208
Name: query hypothyroid, dtype: int64
---- query hyperthyroid ---
f    2912
t     173
Name: query hyperthyroid, dtype: int64
---- lithium ---
f    3067
t      18
Name: lithium, 

Figure 1: Correlations of numeric features


In [16]:
# Visualizing correlations of numeric features
print("Figure 1: Correlations of numeric features")
numeric = hyper_clean[['age', 'TSH', 'TT4', 'T4U', 'FTI']]
cor = numeric.corr()
plt.figure(figsize=(20, 10))
sn.set(font_scale=1)
sn.heatmap(cor, annot=True, cmap=plt.cm.Blues);

In [17]:
print("Figure 2: Realization of class counts")
pd.DataFrame(hyper_clean['binaryClass'].value_counts())

Figure 2: Realization of class counts


,binaryClass
0,3020
1,65


## Model Training & Analysis

In [18]:
# Splitting data
X = hyper_clean.drop(columns="binaryClass")
y = hyper_clean['binaryClass']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.head()

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,query hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH,TT4,T4U,FTI
2290,47.0,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,0.79,117.0,0.85,138.0
2794,25.0,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,1.90,167.0,1.21,137.0
625,66.0,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,2.00,134.0,0.98,137.0
467,57.0,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,1.50,135.0,1.02,132.0
2238,47.0,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,2.60,66.0,0.75,88.0


In [19]:
# Preprocessing data
onehot = ['sex', 'on thyroxine', 'query on thyroxine', 'on antithyroid medication',
          'sick', 'pregnant', 'thyroid surgery', 'I131 treatment', 'query hypothyroid',
          'query hyperthyroid', 'lithium', 'goitre', 'tumor', 'psych', 'hypopituitary']
numeric = ['age', 'TSH', 'TT4', 'T4U', 'FTI']

In [20]:
ct = make_column_transformer(
    (StandardScaler(), numeric),
    (OneHotEncoder(handle_unknown='ignore'), onehot)
)

In [21]:
transformed_X_train = ct.fit_transform(X_train)
transformed_X_test = ct.transform(X_test)
transformed_X_test

array([[-1.10597   , -0.14793839, -0.43759789, ...,  1.        ,
         1.        ,  0.        ],
       [ 0.07210098, -0.15183635, -0.29857674, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.82178251, -0.16353021, -0.46540212, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 1.51791536, -0.1986118 ,  0.67457132, ...,  0.        ,
         1.        ,  0.        ],
       [-1.53435944, -0.17444448, -0.32638097, ...,  0.        ,
         1.        ,  0.        ],
       [-0.73112923, -0.20562812,  2.78769281, ...,  0.        ,
         1.        ,  0.        ]])

In [22]:
# Creating LogisticRegression Classifier
pipe_log = make_pipeline(ct, LogisticRegression(max_iter=1000, C=1))
cv = cross_validate(pipe_log, X_train, y_train, error_score='raise', return_train_score=True)
print("Figure 3: Cross-Validation Scores for the Classifier")
pd.DataFrame(cv)

Figure 3: Cross-Validation Scores for the Classifier


,fit_time,score_time,test_score,train_score
0,0.100318,0.020079,0.993056,0.987840
1,0.072612,0.019636,0.990741,0.987840
2,0.071722,0.016439,0.986111,0.988998
3,0.070975,0.018011,0.981481,0.989577
4,0.075010,0.017949,0.986079,0.989005


Cross validation performs well so this model will be applied to the dataset.

In [23]:
lr = LogisticRegression(max_iter=1000, C=1)
X_train_trans = ct.fit_transform(X_train)
X_test_trans = ct.transform(X_test)
lr.fit(X_train_trans, y_train)
train_preds = lr.predict(X_train_trans)

Figure 4: Visualization of Classification with TSH and TT4 concentration on the axes for training set


Text(0, 0.5, 'TT4 concentration')

In [24]:
print("Figure 4: Visualization of Classification with TSH and TT4 concentration on the axes for training set")
plt.scatter(X_train["TSH"], X_train["TT4"], c=train_preds, s=50, cmap='viridis')
plt.xlabel("TSH concentration")
plt.ylabel("TT4 concentration")

In [25]:
accuracy_score(train_preds, y_train)

0.9879573876794813

The model produces a very strong 98% accuracy rate on the training data. 

In [26]:
# Test set
test_preds = lr.predict(X_test_trans)

Figure 5: Visualization of Clusters with TSH and TT4 concentration on the axes for test set


Text(0, 0.5, 'TT4 concentration')

In [27]:
print("Figure 5: Visualization of Clusters with TSH and TT4 concentration on the axes for test set")
plt.scatter(X_test["TSH"], X_test["TT4"], c=test_preds, s=50, cmap='viridis')
plt.xlabel("TSH concentration")
plt.ylabel("TT4 concentration")

In [28]:
accuracy_score(test_preds, y_test)

0.980561555075594

The model also performs quite well on the test set with a 98% accuracy.

In [29]:
# Confusion Matrix
print("Figure 6: Confusion matrix for test predictions")
pd.DataFrame(confusion_matrix(y_test, test_preds))

Figure 6: Confusion matrix for test predictions


,0,1
0,899,3
1,15,9


## Discussion
The model we trained has a 98% accuracy when tested on a test set. As we were provided by a plethora of measurements specific to thyroid hormones tied to hyperthyroidism, we expected to be able to train a classifier with a high level of accuracy. We believe that this model could act as a further test to backup a doctor's medical opinion for hyperthyroidism. Additionally, due to the lower analysis cost of prediction, this could potentially be an easy resource for people to self-test for the disease. Currently, there are at-home thyroid tests you can administer to measure thyroid hormone levels. By lowering the barrier of entry, more potential hyperthyroidism patients can detect their diseases early on instead of waiting for a medical appointment. A future project could be to model hyperthyroidism disease progression over time. Thyroid hormone levels exist on a spectrum, and simply saying someone has hyperthyroidism or hypothyroidism is severely oversimplifying the disease. Additionally, other projects could look if the dataset holds out in a modern setting. The dataset was published in 1987 and it has been studied that hyperthyroidism increased between 1987 and 1995 due to an increase in salt in food (Mostbeck et al., 1998).


## References

Allahabadia, A. (2000). Age and gender predict the outcome of treatment for Graves' hyperthyroidism. Journal of Clinical Endocrinology & Metabolism, 85(3), 1038–1042. https://doi.org/10.1210/jc.85.3.1038

De Leo, S., Lee, S. Y., & Braverman, L. E. (2016). Hyperthyroidism. The Lancet, 388(10047), 906–918. https://doi.org/10.1016/s0140-6736(16)00278-6

Mostbeck, A., Galvan, G., Bauer, P., Eber, O., Atefie, K., Dam, K., Feichtinger, H., Fritzsche, H., Haydl, H., Köhn, H., König, B., Koriska, K., Kroiss, A., Lind, P., Markt, B., Maschek, W., Pesl, H., Ramschak-Schwarzer, S., Riccabona, G., … Zechmann, W. (1998). The incidence of hyperthyroidism in Austria from 1987 to 1995 before and after an increase in salt iodization in 1990. European Journal of Nuclear Medicine and Molecular Imaging, 25(4), 367–374. https://doi.org/10.1007/s002590050234


U.S. Department of Health and Human Services. (n.d.). Hyperthyroidism (overactive thyroid). National Institute of Diabetes and Digestive and Kidney Diseases. Retrieved February 17, 2023, from https://www.niddk.nih.gov/health-information/endocrine-diseases/hyperthyroidism
